<a href="https://colab.research.google.com/github/SumaOladri/potential-waddle/blob/master/NMT_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import unicodedata
import re
import numpy as np
import os
import io
import time

In [4]:
!pip3 install indic-nlp-library

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 13.9 MB/s eta 0:00:00


In [5]:
#!pip install indic-nlp-library

#!pip3 install indic
#import indic

ModuleNotFoundError: No module named 'indic'

In [6]:
!wget "http://lotus.kuee.kyoto-u.ac.jp/WAT/indic-multilingual/indic_languages_corpus.tar.gz"

--2024-04-06 02:43:36--  http://lotus.kuee.kyoto-u.ac.jp/WAT/indic-multilingual/indic_languages_corpus.tar.gz
Resolving lotus.kuee.kyoto-u.ac.jp (lotus.kuee.kyoto-u.ac.jp)... 130.54.208.131
Connecting to lotus.kuee.kyoto-u.ac.jp (lotus.kuee.kyoto-u.ac.jp)|130.54.208.131|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 132762852 (127M) [application/x-gzip]
Saving to: ‘indic_languages_corpus.tar.gz’

indic_languages_cor 100%[===================>] 126.61M  7.68MB/s    in 13s     

2024-04-06 02:43:50 (9.75 MB/s) - ‘indic_languages_corpus.tar.gz’ saved [132762852/132762852]



In [7]:
import tarfile
with tarfile.open("/content/indic_languages_corpus.tar.gz",'r:gz') as tar:
  tar.extractall()
print('done')

done


In [3]:
pwd

'/content'

In [8]:
%cp indic_languages_corpus/bilingual/hi-en/* .

In [5]:
%rm -r indic_languages_corpus indic_languages_corpus.tar.gz

In [9]:
#Understand how training data looks like
f = open('train.en')
w1 = f.readlines()
print(len(w1))
print(w1[0:5])

84557
['And what is their Sigil?\n', 'I do not want to die.\n', "It's the same country I think.\n", "Then they'll be crying like babies.\n", '- No, I need power up!\n']


In [10]:
h = open('train.hi')
w2 = h.readlines()
print(len(w2))
print(w2[0:5])

84557
['और उनके Sigil क्या है?\n', 'मैं मरना नहीं चाहता.\n', 'यह मुझे लगता है कि एक ही देश है.\n', 'फिर ये नन्हें बच्चों की तरह रोएँगे।\n', 'नहीं, मुझे पावर की जरुरत है !\n']


In [12]:
#from indicnlp import indic_tokenize
#!pip3 install indicnlp.tokenize

In [14]:
NUM_SENTENCES = 70000

In [13]:
from indicnlp.tokenize import indic_tokenize

In [15]:
input_sentences = []
output_sentences = []
count=0
for line in open(r'train.en',encoding='utf-8'):
  count += 1
  if count>NUM_SENTENCES:
    break
  input_sentence = line.rstrip().strip('\n').strip('-')
  input_sentences.append(input_sentence)
count=0
for line in open(r'train.hi'):
  count+=1
  if count>NUM_SENTENCES:
    break
  output_sentence = line.rstrip().strip('\n').strip('-')
  line = indic_tokenize.trivial_tokenize(output_sentence)
  output_sentences.append(['<sos>']+ line + ['<eos>'])

print(type(input_sentences[10]))
print(type(output_sentences[10]))

<class 'str'>
<class 'list'>


In [16]:
print("num samples input:", len(input_sentences))
print("num samples output:", len(output_sentences))

num samples input: 70000
num samples output: 70000


In [17]:
input_sentences[-1]
#output_sentences[-1]

'Her face.'

In [18]:
import re
import unicodedata

In [19]:

def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

In [20]:
def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<sos> ' + w + ' <eos>'
  return w

In [21]:
for i in range(len(input_sentences)):
   input_sentences[i] = preprocess_sentence(input_sentences[i])

print(input_sentences[8])
print(output_sentences[8])

<sos> i told her we rest on sundays . <eos>
['<sos>', 'मैं', 'रविवार', 'को', 'उसे', 'हम', 'बाकी', 'बताया', '.', '<eos>']


In [22]:
def tokenize(lang,pad):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
  lang_tokenizer.fit_on_texts(lang)
  tensor = lang_tokenizer.texts_to_sequences(lang)
  print(lang_tokenizer)
  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')
  return tensor, lang_tokenizer

In [23]:
def load_dataset(inp_lang, targ_lang):
  # creating cleaned input, output pairs
  input_tensor, inp_lang_tokenizer = tokenize(inp_lang, 'post')
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang, 'post')

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [24]:
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(input_sentences, output_sentences)

In [25]:
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]
print(max_length_targ)
print(max_length_inp)

69
72


In [26]:
print(target_tensor[9])

[  1  47 203  18 203  26  39 553  79  29   5 270   8   2   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]


In [27]:
print(input_tensor[9])

[   1    5  106   62   63  462 6235   21    4   59    8    2    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


In [28]:
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2,random_state=7)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

56000 56000 14000 14000


In [32]:
inp_lang.index_word[4]

','

In [33]:
def convert(lang,tensor):
  for t in tensor:
    if t!=0:
      print("%d---->%s" %(t,lang.index_word[t]))
      print("%s----->%d" %(lang.index_word[t],lang.word_index[lang.index_word[t]]))




In [34]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
1----><sos>
<sos>----->1
74---->did
did----->74
5---->you
you----->5
2270---->threaten
threaten----->2270
21---->me
me----->21
8---->?
?----->8
2----><eos>
<eos>----->2

Target Language; index to word mapping
1----><sos>
<sos>----->1
15---->आप
आप----->15
26---->मुझे
मुझे----->26
1426---->धमकी
धमकी----->1426
45---->किया
किया----->45
29---->था
था----->29
8---->?
?----->8
2----><eos>
<eos>----->2


In [37]:
len(inp_lang.word_index)

17618

In [39]:
BUFFER_SIZE = len(input_tensor_train)#---len of Input Tensor ie no of Training points
BATCH_SIZE = 64
steps_per_epoch = BUFFER_SIZE/BATCH_SIZE
embedding_dim = 256
units = 1024 #GRU units
# getting the size of the input and output vocabularies.
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1


dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train,target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE,drop_remainder=True)

print(BUFFER_SIZE)
print(BATCH_SIZE)
print(steps_per_epoch)
print(vocab_inp_size)
print(vocab_tar_size)
print(max_length_targ)
print(max_length_inp)




56000
64
875.0
17619
22224
69
72


In [41]:
# to understand the shape of an input batch
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 72]), TensorShape([64, 69]))

In [44]:
class Encoder(tf.keras.Model):
  def __init__(self,vocab_inp_size,embedding_dim,units,batch_size):
    super(Encoder, self).__init__()
    self.batch_size = batch_size
    self.vocab_inp_size = vocab_inp_size
    self.units = units
    self.embedding = tf.keras.layers.Embedding(vocab_inp_size,embedding_dim)
    self.gru = tf.keras.layers.GRU(self.units,return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform') # define the GRU layer)
  def call(self, x, hidden): # this function is invoked when the function encoder is called with an input and an initialised hidden layer
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden) # pass input x into the GRU layer
    return output, state # function returns the encoder output and the hidden state
  def initialize_hidden_state(self): #intialise hidden layer to all zeroes (for determining the shape)
    return tf.zeros((self.batch_size, self.units))

In [50]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE) # create an Encoder class object

# sample input to get a sense of the shapes.
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 72, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [46]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_tar_size, embedding_dim, units, batch_size):
    super(Decoder, self).__init__()
    self.batch_sz = batch_size # batch_size which is defined as 64
    self.dec_units = units # the number of decoder GRU units
    self.vocab_size = vocab_inp_size
    self.embedding = tf.keras.layers.Embedding(vocab_inp_size, embedding_dim) # defining an embedding layer for the target language output.
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform') # GRU layer
    self.fc = tf.keras.layers.Dense(vocab_inp_size)


  def call(self, x, hidden):

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x) # creating an embedding layer for the target output

    # passing the initial state to the GRU as the hidden state
    output, state = self.gru(x, initial_state=hidden)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output) # pass the output through the dense layer

    return x, state # return decoder output and decoder state

In [48]:
#max_length_targ, max_length_inp
vocab_inp_size

17619

In [49]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (64, 22224)


In [52]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none') #Loss function is categorical crossentropy

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [51]:
pwd

'/content'

In [54]:
checkpoint_dir = '/content/tutorial_checkpoint_nmt'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [55]:
@tf.function
def train_step(inp,targ,enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output,enc_hidden = encoder(inp,enc_hidden)
    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<sos>']]*BATCH_SIZE,1)

    for t in range(1,targ.shape[1]):
      predictions, dec_hidden = decoder(dec_input, dec_hidden)
      loss += loss_function(targ[:, t], predictions)

      dec_input = tf.expand_dims(targ[:, t], 1)
  batch_loss = loss/int(targ.shape[1])
  variables = encoder.trainable_variables + decoder.trainable_variables
  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables)) # doing gradient descent

  return batch_loss





In [57]:
#targ.shape

In [1]:
train = True
EPOCHS=5
if train:
  for epoch in range(EPOCHS):
     start = time.time()
     enc_hidden = encoder.initialize_hidden_state()
     total_loss = 0

     for (batch, (inp, targ)) in enumerate(dataset.take(int(steps_per_epoch))):

        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

        if batch % 100 == 0:
          print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                    batch,
                                                    batch_loss.numpy()))

    # saving (checkpoint) the model every 2 epochs
     if (epoch + 1) % 2 == 0:
       checkpoint.save(file_prefix = checkpoint_prefix)

     print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                          total_loss / steps_per_epoch))
     print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

NameError: name 'time' is not defined